In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
data = pd.read_csv('train.csv')

In [ ]:
data.head()

In [ ]:
data = np.array(data)
m,n = data.shape
np.random.shuffle(data)

data_dev= data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]

data_train = data[1000:].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train/255




In [ ]:
# print(Y_train)
# print(X_train)
print(X_train.shape)
print(X_train[0].shape)
print(X_train[:,0].shape)

In [ ]:
def init_params():
    W1 = np.random.rand(10, 784)-0.5
    B1 = np.random.rand(10, 1)-0.5
    W2 = np.random.rand(10, 10)-0.5
    B2 = np.random.rand(10, 1)-0.5
    return W1, B1, W2 , B2

def relU(z):
    return np.maximum(0,z)

def softmax(z):
    exp = np.exp(z - np.max(z))
    return exp / exp.sum(axis=0)

def forward_prop(w1, b1, w2, b2 , x):
    Z1 = w1.dot(x) +b1
    A1 = relU(Z1)
    
    Z2 = w2.dot(A1)+b2
    a2 = softmax(Z2)
    return Z1, A1, Z2, a2
    
    
def one_hot(Y):
    one_hot_y = np.zeros((Y.size, Y.max()+1))
    one_hot_y[np.arange(Y.size), Y]=1
    one_hot_y = one_hot_y.T
    return one_hot_y

def deriv_relu(Z):
    return Z>0
    
def back_prop(z1, a1, z2, a2, w2, x, y):
    m = y.size
    one_hot_y = one_hot(y)
    dz2 = a2-one_hot_y
    dw2 = 1/m * dz2.dot(a1.T)
    db2 = 1/m * np.sum(dz2)
    dz1 = w2.T.dot(dz2)* deriv_relu(z1)
    dw1 = 1/m *dz1.dot(x.T)
    db1 = 1/m* np.sum(dz1)
    
    return dw1, db1, dw2 , db2

def update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, aplha):
    w1 = w1-aplha*dw1
    b1 = b1-aplha*db1
    w2 = w2-aplha*dw2
    b2 = b2-aplha*db2
    return w1, b1, w2, b2

In [ ]:
def get_predictions(a2):
    return np.argmax(a2, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions==Y)/Y.size

def gradient_descent(X, Y, iterations, alpha):
    w1, b1, w2, b2 = init_params()
    for i in range(iterations):
        z1, a1, z2, a2 = forward_prop(w1, b1, w2, b2, X)
        dw1 , db1, dw2, db2 = back_prop(z1, a1, z2, a2, w2, X, Y)
        w1, b1, w2, b2 = update_params(w1, b1, w2, b2, dw1, db1, dw2, db2, alpha)
        if(i%10==0):
            print('iterations: ', i)
            print("accuracy: ", get_accuracy(get_predictions(a2), Y))
    return w1, b1, w2, b2

In [ ]:
w1, b1, w2, b2 = gradient_descent(X_train , Y_train , 500, 0.1)

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
for i in range(50):
    test_prediction(i, w1, b1, w2, b2)
